In [ ]:
%pip install transformers datasets accelerate evaluate rouge_score bitsnbites

In [ ]:
from huggingface_hub import login
login()

In [ ]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM, AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

In [ ]:
dataset = load_dataset("MattBoraske/reddit-AITA-binary-submissions-and-comments-top-2k")

In [ ]:
hf_repo = "MattBoraske/flan-t5-xl-reddit-AITA-binary-top-2k"

In [ ]:
bnb_config = BitsAndBytesConfig(
  load_in_4bit=True,
  bnb_4bit_quant_type='nf4',
  bnb_4bit_compute_dtype='float16',
  bnb_4bit_use_double_quant=False,
)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(hf_repo)
model = AutoModelForSeq2SeqLM.from_pretrained(
  hf_repo,
  device_map='auto',
  quantization_config=bnb_config,
  torch_dtype=torch.bfloat16
)

In [ ]:
from model_evaluator import Model_Evaluator

In [ ]:
output_file = 'flan-t5-xl-reddit-AITA-binary-predictions.json'
submission_texts, predictions, references, predicted_classes, correct_classes, ambiguity_scores = Model_Evaluator.get_model_predictions(model, tokenizer, dataset['test'].select(range(10)), output_file)

In [ ]:
classification_type = 'binary'
output_files = [
    'classification_report.json',
    ('Flan-T5-XL-Reddit-AITA-Binary-Top-2k Classifications', 'confusion_matrix.png'),
    'mcc.json'
]

In [ ]:
Model_Evaluator.evaluate_model(predictions, references, predicted_classes, correct_classes, ambiguity_scores, classification_type, output_files)